In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors, LocalOutlierFactor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
import nfl_data_py as nfl
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
today = dt.date.today()
year = today.year
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

In [17]:
df = nfl.import_schedules(years=range(year-5,year))

In [18]:
# Prepare dataframe by dropping irrelevant predictors and formatting columns for KNN
df['Under'] = np.where(df['total'] < df['total_line'], 1, 0)
df['Push'] = np.where(df['total'] == df['total_line'], 1, 0)
df = df[df.Push != 1]

def date_to_month(time_str):
    year, month, day = map(int, time_str.split('-'))
    return month
df['month'] = df['gameday'].apply(date_to_month)
# Function to convert time to seconds
def time_to_seconds(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours * 3600 + minutes * 60
# Apply the function to the 'time' column
df['gametime'] = df['gametime'].apply(time_to_seconds)

dict_day = {"weekday": {"Sunday": 0, "Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5, "Saturday": 6}}
df.replace(dict_day, inplace=True)
dict_roof = {"roof": {"outdoors": 0, "dome": 1, "closed": 2, "open": 3}}
df.replace(dict_roof, inplace=True)
dict_surface = {"surface": {"grass": 0, "grass ": 0, "fieldturf": 1, "astroturf": 2, "sportturf": 3, "matrixturf": 4, "astroplay": 5, "a_turf": 6, "dessograss": 7}}
df.replace(dict_surface, inplace=True)

df = pd.get_dummies(df, drop_first=True, columns=['game_type', 'location', 'stadium_id', 'home_team', 'away_team'])
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [19]:
df.columns
features = df.drop(['Under', 'Push', 'gameday', 'game_id', 'home_score', 'away_score', 'result', 'total', 'overtime', 'old_game_id', 'gsis', 'nfl_detail_id', 'pfr', 'pff', 'espn', 'ftn', 'away_qb_id', 'home_qb_id', 'away_qb_name', 'home_qb_name', 'away_coach', 'home_coach', 'referee', 'stadium', 'wind', 'temp'], axis=1).columns

# Accuracy Testing

In [20]:
# Model building
y = df.Under
X = df[features]

precis_array = []
acc_array = []
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score
for i in range(1, 51):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=i, stratify=y)
    for j in range(1,11):
        rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=j)
        rf.fit(X_train, y_train)
        preds = rf.predict(X_test)
        acc = accuracy_score(y_test, preds)
        precis = precision_score(y_test, preds)
        precis_array.append(precis)
        acc_array.append(acc)
# combined = pd.DataFrame(dict(actual=y_test, prediction=preds))
# pd.crosstab(index=combined['actual'], columns=combined['prediction'])

In [21]:
def find_mean(array):
    if len(array) == 0:
        return 0  # To handle empty arrays
    return sum(array) / len(array)
find_mean(precis_array)

0.5964086023709899

In [22]:
find_mean(acc_array)

0.5466222222222221

# Weekly Plays

In [34]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import nfl_data_py as nfl
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
today = dt.date.today()
year = today.year

# Variable declaration
df = nfl.import_schedules(years=range(year-3,year+1))
currSeason = df[df.season == year]
predWeek = currSeason[['week', 'total']].dropna()
if np.isnan(predWeek.week.max()):
    predWeek = 1
else:
    predWeek = predWeek.week.max() + 1
#predWeek=1
# Prepare dataframe by dropping irrelevant predictors and formatting columns for KNN
df['Under'] = np.where(df['total'] < df['total_line'], 1, 0)
df['Push'] = np.where(df['total'] == df['total_line'], 1, 0)
df = df[df.Push != 1]

def date_to_month(time_str):
    year, month, day = map(int, time_str.split('-'))
    return month
df['month'] = df['gameday'].apply(date_to_month)
# Function to convert time to seconds
def time_to_seconds(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours * 3600 + minutes * 60
# Apply the function to the 'time' column
df['gametime'] = df['gametime'].apply(time_to_seconds)

dict_day = {"weekday": {"Sunday": 0, "Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4, "Friday": 5, "Saturday": 6}}
df.replace(dict_day, inplace=True)
dict_roof = {"roof": {"outdoors": 0, "dome": 1, "closed": 2, "open": 3}}
df.replace(dict_roof, inplace=True)
dict_surface = {"surface": {"grass": 0, "grass ": 0, "fieldturf": 1, "astroturf": 2, "sportturf": 3, "matrixturf": 4, "astroplay": 5, "a_turf": 6, "dessograss": 7}}
df.replace(dict_surface, inplace=True)

df = pd.get_dummies(df, drop_first=True, columns=['game_type', 'location', 'stadium_id', 'home_team', 'away_team'])

features = df.drop(['Under', 'Push', 'gameday', 'game_id', 'home_score', 'away_score', 'result', 'total', 'overtime', 'old_game_id', 'gsis', 'nfl_detail_id', 'pfr', 'pff', 'espn', 'ftn', 'away_qb_id', 'home_qb_id', 'away_qb_name', 'home_qb_name', 'away_coach', 'home_coach', 'referee', 'stadium', 'wind', 'temp'], axis=1).columns

train_df = df[(df.season < year) | ((df.season == year) & (df.week < predWeek))]
test_df = df[(df.season == year) & (df.week == predWeek)]
train_df.dropna(inplace=True)
X_train = train_df[features]
y_train = train_df.Under
X_test = test_df[features]
y_test = test_df.Under

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
rf.fit(X_train, y_train)
preds = rf.predict(X_test)
X_test['Prediction'] = preds
# Predicted Plays log
nextPlays = pd.merge(right=X_test, left=currSeason, right_index=True, left_index=True, how='left')
nextPlays = nextPlays[nextPlays.Prediction == 1]
nextPlays = nextPlays[['game_id', 'season_x', 'week_x', 'home_team', 'away_team', 'gametime_x', 'weekday_x', 'total_line_x', 'under_odds_x']]
nextPlays.columns = ['Game ID', 'Season', 'Week', 'Home', 'Away', 'Start Time', 'Day', 'Total Line', 'Under Odds']

# Value cleanup
dict_day = {"Day": {0: "Sunday", 1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday"}}
nextPlays.replace(dict_day, inplace=True)
nextPlays

,Game ID,Season,Week,Home,Away,Start Time,Day,Total Line,Under Odds
6706,2024_01_BAL_KC,2024,1,KC,BAL,20:20,Thursday,46.0,-110.0
6708,2024_01_PIT_ATL,2024,1,ATL,PIT,13:00,Sunday,43.0,-115.0
6709,2024_01_ARI_BUF,2024,1,BUF,ARI,13:00,Sunday,46.0,-112.0
6710,2024_01_TEN_CHI,2024,1,CHI,TEN,13:00,Sunday,43.0,-110.0
6711,2024_01_NE_CIN,2024,1,CIN,NE,13:00,Sunday,40.5,-112.0
6712,2024_01_HOU_IND,2024,1,IND,HOU,13:00,Sunday,49.0,-112.0
6714,2024_01_CAR_NO,2024,1,NO,CAR,13:00,Sunday,41.5,-112.0
6715,2024_01_MIN_NYG,2024,1,NYG,MIN,13:00,Sunday,42.5,-108.0
6716,2024_01_LV_LAC,2024,1,LAC,LV,16:05,Sunday,41.0,-108.0
6718,2024_01_DAL_CLE,2024,1,CLE,DAL,16:25,Sunday,42.0,-108.0
